In [44]:
# This file was basically a sliiightly updated expansion.ipynb, which I deleted
# I think this one was an attempt to augment my collected API data, but failed
# (because I broke the API rate limits accidentally and got errors.. hope it's fixed?!)

# Current goal for this file (possibly misnamed) is to more fully plumb the API for more data 
# and eventually even auto-label the tags?

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import epicstore_api
from epicstore_api import EpicGamesStoreAPI, OfferData
import json

In [45]:
og_small = pd.read_csv("df_scraped.csv")
api = EpicGamesStoreAPI()
og_small
# also use dff for joined, first try data (full names/same index, but 25% missing api stuff)
# use primarily for names only, so it's fine

Unnamed: 0  Number                         Name       Start         End  \
0             0     477                   The Bridge   3/14/2024   3/21/2024   
1             1     476     Deus Ex: Mankind Divided   3/14/2024   3/21/2024   
2             2     475                 Astro Duel 2    3/7/2024   3/14/2024   
3             3     474  Aerial_Knight’s Never Yield   2/29/2024    3/7/2024   
4             4     473       Super Meat Boy Forever   2/22/2024   2/29/2024   
..          ...     ...                          ...         ...         ...   
472         472       5                  Axiom Verge    2/7/2019   2/22/2019   
473         473       4       The Jackbox Party Pack   1/24/2019    2/7/2019   
474         474       3  What Remains of Edith Finch   1/11/2019   1/24/2019   
475         475       2               Super Meat Boy  12/28/2018   1/10/2019   
476         476       1                   Subnautica  12/12/2018  12/27/2018   

                                                  Link  
0       https://store.epicgames.com/en-US/p/the-bridge  
1    https://store.epicgames.com/en-US/p/deus-ex-ma...  
2    https://store.epicgames.com/en-US/p/astro-duel...  
3    https://store.epicgames.com/en-US/p/aerial-kni...  
4    https://store.epicgames.com/en-US/p/super-meat...  
..                                                 ...  
472    https://store.epicgames.com/en-US/p/axiom-verge  
473  https://store.epicgames.com/en-US/p/jackbox-pa...  
474  https://store.epicgames.com/en-US/p/what-remai...  
475  https://store.epicgames.com/en-US/p/super-meat...  
476     https://store.epicgames.com/en-US/p/subnautica  

[477 rows x 6 columns]

In [49]:
# test bench and helper test functions off of saved CSV data

# loads the csv into "dff"
dff = pd.read_csv("df_joined.csv")

def titleresults(title, trycounts = 10):
    # returns TITLES ONLY for a default 10 search
    resdict = api.fetch_store_games(count = trycounts, keywords = title)
    match_found = 0
    for element in resdict["data"]["Catalog"]["searchStore"]["elements"]:
        if element["title"] == title:
            print("\n\nMATCH FOUND!!!!\n\n")
            match_found = 1
        print(element["title"])
    return match_found

def getname(index_num):
    # grabs the full name from index to save me typing
    return (dff.iloc[index_num]["Name"])

def jprint(thing):
    print(json.dumps(thing, indent = 2, default = str))

def testsuite(index_num):
    name = getname(index_num)
    print("-> Testing", name)
    print("A search returns the following:", titleresults(name))
    if titleresults(name) == 1:
        jprint(get_gamedata(name))
    elif titleresults(name, 100) == 1:
        jprint(get_gamedata(name))
    else:
        print("\n\nsadface") # somethign was supposed to go here

In [47]:
# setting up a wrapped function to test other titles, eventually for the whole table, but to respect rate limits
def single_get(name, sleep_time = .7):
    time.sleep(sleep_time)
    #TODO: POTENTIALLY UNCOMMENT BELOW TEST OUTPUT PRINT STATEMENTS
    print("Attempting to grab data about", name)
    # No additional text = Success
    try:    
        done = get_better_gamedata(name)
    #TODO: trying to think of possibility get_gamedata fails... maybe add a better fail return?
    except:
        try:
            # in this case, possibly the name wasn't within the first 5 results, so trying more and 
            print("RETRY RETRY:", name)
            done = get_better_gamedata(name, 40, True)
        except:
            # on a fail, I will return None and I will have to make this into an empty row later
            print("\tFAILED RETRY")
            done = None
    return done

# The following is superior enough to be the new default
def get_better_gamedata(gamename, searchsize = 5, less_strict = False):
    # takes a string gamename, searches for it (defaults to 5 search results), finds exact name/title match
    # returns dict with id, descr[iption], namespace, orig[inal]_price, fmt_orig_price (nicely formatted), and tags (list)
    dict = api.fetch_store_games(count = searchsize, keywords = gamename)
    match = None
    for element in dict["data"]["Catalog"]["searchStore"]["elements"]:
        # ideal case
        if element["title"] == gamename:
            match = element
            break

    # less strict matching mode, if enabled
    # generally want to change the api to match the og df name
    # will re-loop through returned results a second time looking for a match
    if less_strict == True:
        for element in dict["data"]["Catalog"]["searchStore"]["elements"]:
            # api space-hyphens become : like og df only, should never false positive
            if re.sub(r" -", ":", element["title"]) == gamename:
                match = element
                break
            # get rid of ALL nonalphanumerics, including special characters and whitespace
            # ALSO checks lowercase for case issues at same time
            if re.sub(r'[^a-zA-Z0-9]', '', element["title"]).lower() == re.sub(r'[^a-zA-Z0-9]', '', gamename).lower():
                match = element
                break
        # last resort, does ALL the dataframe version match the START of the api version? If so use the first result
        for element in dict["data"]["Catalog"]["searchStore"]["elements"]:
            if element["title"][:len(gamename)] == gamename:
                match = element
                break

    # NEEDED ONLY IF not wrapped in single_get (which handles errors)
    # if match == None:
    #     return {}

    jprint(match)
    
    return_dict = {
    "top_id": match["id"],
    "top_namespace": match["namespace"],
    "descr": match["description"],
    "items_id": match["items"]["id"],
    "items_namespace": match["items"]["namespace"],
    # "items_id": match["items"][0],
    # "items_namespace": match["items"][1],
    "orig_price": match["price"]["totalPrice"]["originalPrice"],
    "fmt_orig_price": match["price"]["totalPrice"]["fmtPrice"]["originalPrice"],
    "tags": match["tags"],  # list but looks like individual dicts
    "seller": match["seller"]["name"],
    "prod_slug": match["productSlug"],
    "url_slug": match["urlSlug"]
    #TODO: possibly format the tags better
    }
    return return_dict

In [55]:

# testsuite(22)
# jprint(api.fetch_store_games(count = 1, keywords = getname(2)))
astro = api.fetch_store_games(count = 1, keywords = getname(2))
# jprint(astro["id"])
# jprint(astro)

# astro["data"]["Catalog"]["searchStore"]["elements"][0]["items"]#[0]["items"][0]

for element in astro["data"]["Catalog"]["searchStore"]["elements"]:
    # ideal case
    if element["title"] == getname(2):
        match = element
        break

jprint(match)
type(match)

return_dict = {
    "top_id": match["id"],
    "top_namespace": match["namespace"],
    "descr": match["description"],
    "items_id": match["items"]["id"],
    "items_namespace": match["items"]["namespace"],
    # "items_id": match["items"][0],
    # "items_namespace": match["items"][1],
    "orig_price": match["price"]["totalPrice"]["originalPrice"],
    "fmt_orig_price": match["price"]["totalPrice"]["fmtPrice"]["originalPrice"],
    "tags": match["tags"],  # list but looks like individual dicts
    "seller": match["seller"]["name"],
    "prod_slug": match["productSlug"],
    "url_slug": match["urlSlug"]}

# spacer = api.fetch_store_games(count = 10, keywords = getname(22))
# # print(spacer["id"])
# jprint(spacer)


{
  "title": "Astro Duel 2",
  "id": "bc38eac0277d41ec955690e25779cb53",
  "namespace": "4a681a809a094e2c8dcc68353c68fed6",
  "description": "Astro Duel 2 is a sci-fi combat game combining top-down space dog fights with up-close platforming action. Whether versus or co-op, battle through space and on foot at the same time! Strike within the fully-destructible arenas or just nuke \u2018em from orbit.",
  "effectiveDate": "2024-03-07T16:00:00.000Z",
  "keyImages": [
    {
      "type": "OfferImageWide",
      "url": "https://cdn1.epicgames.com/spt-assets/3251b927449a471c883aab00c076d63c/astro-duel-2-ukpi8.png"
    },
    {
      "type": "OfferImageTall",
      "url": "https://cdn1.epicgames.com/spt-assets/3251b927449a471c883aab00c076d63c/astro-duel-2-1w3a4.png"
    },
    {
      "type": "Thumbnail",
      "url": "https://cdn1.epicgames.com/spt-assets/3251b927449a471c883aab00c076d63c/astro-duel-2-1w3a4.png"
    }
  ],
  "seller": {
    "id": "o-bkftz4vh5gcrv3aks6zkrtkulrgctm",
    "name"

TypeError: list indices must be integers or slices, not str

In [13]:
# TAKES A LONG TIME TO RUN
# expand_df = [single_get(a_game) for a_game in dff["Name"]]  # pulls all strings from "Name" column in gdf, same order
# expand_df_copy = expand_df

Attempting to grab data about The Bridge
RETRY RETRY: The Bridge
	FAILED RETRY
Attempting to grab data about Deus Ex: Mankind Divided
RETRY RETRY: Deus Ex: Mankind Divided
	FAILED RETRY
Attempting to grab data about Astro Duel 2
RETRY RETRY: Astro Duel 2
	FAILED RETRY
Attempting to grab data about Aerial_Knight’s Never Yield
RETRY RETRY: Aerial_Knight’s Never Yield
	FAILED RETRY
Attempting to grab data about Super Meat Boy Forever
RETRY RETRY: Super Meat Boy Forever
	FAILED RETRY
Attempting to grab data about Dakar Desert Rally
RETRY RETRY: Dakar Desert Rally
	FAILED RETRY


KeyboardInterrupt: 

In [20]:
# assembles a dataframe from the large list of individual entries
expand_df = [d if d is not None else {} for d in expand_df]
expand_df = pd.DataFrame(expand_df)
expand_df.head()
expand_df.info
# adds in the extra, new information into a merged final frame
og_small.reset_index()
expand_df.reset_index()
df_exp = pd.merge(og_small, expand_df, left_index = True, right_index = True)
df_exp.head()

0
0  0

In [32]:
# not sure what the difference is here, but this appears to do the same thing but actually works?
edf = expand_df_copy.copy()
edf = [d if d is not None else {} for d in edf]
edf = pd.DataFrame(edf)
edf.reset_index()
edf
og_small.reset_index()
all = pd.merge(og_small, edf, left_index = True, right_index = True)
all
# all.to_csv("~/Desktop/dshw/EGS_project/all.csv", index = True)

Unnamed: 0  Number                         Name       Start         End  \
0             0     477                   The Bridge   3/14/2024   3/21/2024   
1             1     476     Deus Ex: Mankind Divided   3/14/2024   3/21/2024   
2             2     475                 Astro Duel 2    3/7/2024   3/14/2024   
3             3     474  Aerial_Knight’s Never Yield   2/29/2024    3/7/2024   
4             4     473       Super Meat Boy Forever   2/22/2024   2/29/2024   
..          ...     ...                          ...         ...         ...   
472         472       5                  Axiom Verge    2/7/2019   2/22/2019   
473         473       4       The Jackbox Party Pack   1/24/2019    2/7/2019   
474         474       3  What Remains of Edith Finch   1/11/2019   1/24/2019   
475         475       2               Super Meat Boy  12/28/2018   1/10/2019   
476         476       1                   Subnautica  12/12/2018  12/27/2018   

                                                  Link  \
0       https://store.epicgames.com/en-US/p/the-bridge   
1    https://store.epicgames.com/en-US/p/deus-ex-ma...   
2    https://store.epicgames.com/en-US/p/astro-duel...   
3    https://store.epicgames.com/en-US/p/aerial-kni...   
4    https://store.epicgames.com/en-US/p/super-meat...   
..                                                 ...   
472    https://store.epicgames.com/en-US/p/axiom-verge   
473  https://store.epicgames.com/en-US/p/jackbox-pa...   
474  https://store.epicgames.com/en-US/p/what-remai...   
475  https://store.epicgames.com/en-US/p/super-meat...   
476     https://store.epicgames.com/en-US/p/subnautica   

                                   id  \
0    6b228a1ad940496dabc89ec6640107f3   
1    a74220fd07e34b01987fb801433fb2e6   
2    bc38eac0277d41ec955690e25779cb53   
3    2fecb1793ce446408cd869611400a797   
4    ef66029a1c0d467ea7755f6a88b4088c   
..                                ...   
472  15a6c7220ba045448aeaed7e7c2f9a1f   
473  f2546a46804147bfb6ab74937caf9295   
474  b1f4888a7c9b4449b6bfbace4de24a9c   
475  13434c6d49b742c7b195201573c146c9   
476  a6d73af1b7de4ae086e61f462a351272   

                                                 descr  \
0                                           The Bridge   
1    Secure the Digital Deluxe edition now to gain ...   
2    Astro Duel 2 is a sci-fi combat game combining...   
3    Take the role of Wally. A mysterious character...   
4                               Super Meat Boy Forever   
..                                                 ...   
472                                        Axiom Verge   
473  For 1-100 players! Your phones or tablets are ...   
474  What Remains of Edith Finch is a collection of...   
475                                     Super Meat Boy   
476                                         Subnautica   

                            namespace  orig_price fmt_orig_price  \
0                                epic       999.0          $9.99   
1    5c5b2b01c8ce475d86cdd2c76fd11f7b      4499.0         $44.99   
2    4a681a809a094e2c8dcc68353c68fed6      1999.0         $19.99   
3    2ed7ed64d8604265924b85bbd5670220      1199.0         $11.99   
4                                epic      1999.0         $19.99   
..                                ...         ...            ...   
472                              epic      1999.0         $19.99   
473                          feverfew      2499.0         $24.99   
474                            badger      1999.0         $19.99   
475                              epic      1499.0         $14.99   
476                              epic      2999.0         $29.99   

                                                  tags  \
0    [{'id': '1298'}, {'id': '1370'}, {'id': '9547'...   
1    [{'id': '1336'}, {'id': '1370'}, {'id': '9547'...   
2    [{'id': '1216'}, {'id': '21894'}, {'id': '1984...   
3    [{'id': '1216'}, {'id': '21894'}, {'id': '9547...   
4    [{'id': '1216'}, {'id': '1370'}, {'id': '9547